In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-04"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
13442,2024-03-04,Polônia 1. Liga,14:00,Niedzwiadki Chemart Przemysl,Slask Wroclaw II,2.45,1.51,173.5,1.85,1.85,0.0,0.00,0.00,WMwaH8ZS,0.408163,0.662252,0.540541,0.540541,0.070415,206.350,67.300165,0.326146,0.0,0.000000,NaN,127.75,2.750,0.921819,0.335207,-38.0,146.990,44.369555,0.301854,1.8,1.643168,0.912871,199.24,1.676,0.740122,0.441600,3.0,73,68,1.75,2.93,176.368,185.646,0.335697,0.000000,NaN,-5.00,-1.000,-1.450000,0.471113,0.2,-0.271113,0.38,0.076,6.710526,0.529264,0.4,-0.129264
13443,2024-03-04,Lituânia Moteru Lyga Feminina,11:00,Panevezys F,Klaipedos Neptunas F,11.60,26.00,139.5,1.85,1.85,0.0,0.00,0.00,bsPudp6Q,0.086207,0.038462,0.540541,0.540541,-0.875332,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,65,79,23.00,1.47,0.000,0.000,0.541614,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
13444,2024-03-04,Lituânia Lkl,13:50,Lietkabelis,Neptunas,1.48,2.53,164.5,1.86,1.80,0.0,0.00,0.00,ObmfElHo,0.675676,0.395257,0.537634,0.555556,0.070933,126.108,28.009026,0.222103,1.8,1.643168,0.912871,113.16,1.526,0.434085,0.284460,5.0,208.208,124.041252,0.595756,0.6,1.341641,2.236068,417.00,2.638,1.793299,0.679795,-37.0,92,75,1.23,5.56,0.000,269.952,0.370305,0.023184,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,5.35,1.070,1.429907,0.000000,0.0,0.000000
13445,2024-03-04,Israel Superliga,13:00,Hapoel Eilat,Maccabi Tel Aviv,5.58,1.13,172.5,1.87,1.87,0.0,0.00,0.00,z3hxVQTd,0.179211,0.884956,0.534759,0.534759,0.064167,372.764,258.682012,0.693957,1.8,1.643168,0.912871,194.40,4.726,3.383435,0.715919,-15.0,110.294,19.679555,0.178428,3.0,0.000000,0.000000,116.16,1.212,0.189658,0.156483,72.0,72,88,2.70,1.32,164.602,128.196,0.937891,0.000000,NaN,-0.59,-0.118,-38.813559,0.000000,0.0,0.000000,0.37,0.074,1.756757,0.000000,0.0,0.000000
13446,2024-03-04,Eua Ncaa,23:00,Montana,Idaho State,1.30,3.38,142.5,1.80,1.91,0.0,0.00,0.00,x4zRGFmn,0.769231,0.295858,0.555556,0.523560,0.065089,122.226,44.163498,0.361327,2.4,1.341641,0.559017,116.00,1.534,0.537429,0.350345,48.0,176.908,107.876984,0.609791,0.6,1.341641,2.236068,139.83,2.292,1.234087,0.538432,4.0,80,79,1.45,1.77,98.550,286.724,0.628539,0.041931,NaN,-0.15,-0.030,-10.000000,0.760873,0.8,0.039127,3.45,0.690,3.449275,0.333831,0.3,-0.033831
13447,2024-03-04,Grécia Liga De Basquete,12:15,Marousi,Apollon Patron,1.28,3.62,158.5,1.81,1.85,0.0,0.00,0.00,UJhqPor0,0.781250,0.276243,0.552486,0.540541,0.057493,258.106,146.760937,0.568607,0.6,1.341641,2.236068,111.47,3.388,2.040581,0.602297,-49.0,459.870,201.360093,0.437863,0.6,1.341641,2.236068,333.20,7.204,3.908008,0.542478,-79.0,71,70,1.57,4.76,325.514,398.238,0.675359,0.015456,NaN,-3.04,-0.608,-0.460526,0.000000,0.0,0.000000,-5.00,-1.000,-2.620000,0.000000,0.0,0.000000
13448,2024-03-04,Europa Liga Aba,15:00,Buducnost,Igokea,1.08,6.79,155.5,1.82,1.84,0.0,0.00,0.00,2yJgQvwC,0.925926,0.147275,0.549451,0.543478,0.073201,148.220,43.133324,0.291009,3.0,0.000000,0.000000,116.64,1.690,0.532870,0.315307,91.0,186.720,66.923527,0.358416,1.2,1.643168,1.369306,272.48,2.776,1.481513,0.533686,-67.0,81,52,1.44,5.24,128.106,252.544,1.026069,0.007728,NaN,-0.23,-0.046,-1.739130,0.000000,0.0,0.000000,-2.68,-0.536,-10.802239,0.000000,0.

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
13442,14:00,Polônia 1. Liga,Niedzwiadki Chemart Przemysl,Slask Wroclaw II,2.45,Back Home
13471,14:00,Ásia Wasl,Al Muharraq,Al-Shabab,1.64,Back Home
13487,21:30,Eua Ncaa,Howard,Delaware State,1.55,Back Home
13488,21:30,Eua Ncaa,Incarnate Word,SE Louisiana,2.47,Back Home
13492,21:30,Eua Ncaa,Prairie View A&M,Jackson State,2.35,Back Home
